In [ ]:
import pandas as pd

from src.database import read_sql

In [ ]:
sql_sickness_abbr = '''
    SELECT neu_kurz as sickness_abbr, string_agg(sdienst_lang, ', ') as description FROM spxdb_archiv_2023.sdienst_mapping
    WHERE sdienst_lang ILIKE('%krank%') AND sdienst_lang NOT ILIKE('%nicht krank%')
    GROUP BY sickness_abbr
'''
read_sql(sql_sickness_abbr)

In [ ]:
sql_sickness_plan = f'WITH mapping AS ({sql_sickness_abbr})' + '''
    SELECT *
    FROM mapping JOIN spxdb_archiv_2023.monatsplan_adapted AS plan
    ON mapping.sickness_abbr = plan.sdienst_kurz_adapted
    WHERE plan.mpebene = 1 -- filter for "ist" (vs. e.g. "soll")
'''
read_sql(sql_sickness_plan + ' LIMIT 10')

In [ ]:
sql_sickness_planteil = f'WITH plan AS ({sql_sickness_plan})' + '''
    SELECT *
    FROM plan JOIN spxdb_archiv_2023.monatsplanteil AS planteil
    ON plan.magrpnr_f = planteil.planteil_nr
'''
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(read_sql(sql_sickness_planteil + ' LIMIT 10'))